In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/siamesenet/BHSig260.zip

In [ ]:
!pip install tensorflow-gpu==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 61.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
    Uninstalling tensorboard-data-server-0.7.1:
      Successfully uninstalled tensorboard-data-server-0.7.1
  Attempting uninstall: google-auth-oauthlib
    Found existing ins

In [ ]:
import pandas as pd
import numpy as np
import tensorflow

import sys
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time
import itertools
import random

from sklearn.utils import shuffle

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout,GlobalAveragePooling2D
from tensorflow.keras.models import Model

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform

from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
path = "/content/BHSig260/Hindi/"

In [ ]:
dir_list = next(os.walk(path))[1]
dir_list.sort()

In [ ]:
orig_groups, forg_groups = [], []
for directory in dir_list:
    images = os.listdir(path+directory)
    images.sort()
    images = [path+directory+'/'+x for x in images]
    break
images

['/content/BHSig260/Hindi/001/H-S-1-F-01.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-02.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-03.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-04.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-05.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-06.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-07.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-08.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-09.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-10.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-11.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-12.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-13.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-14.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-15.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-16.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-17.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-18.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-19.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-20.tif',
 '/content/BHSig260/Hindi/001/H-S-1-F-21.tif',
 '/content/BH

In [ ]:
orig_groups, forg_groups = [], []
for directory in dir_list:
    images = os.listdir(path+directory)
    images.sort()
    images = [path+directory+'/'+x for x in images]
    forg_groups.append(images[:30])
    orig_groups.append(images[30:])

In [ ]:
orig_lengths = [len(x) for x in orig_groups]
forg_lengths = [len(x) for x in forg_groups]

In [ ]:
orig_train, orig_val, orig_test = orig_groups[:120], orig_groups[120:140], orig_groups[140:]
forg_train, forg_val, forg_test = forg_groups[:120], forg_groups[120:140], forg_groups[140:]

In [ ]:
img_h, img_w = 155, 220

In [ ]:
def generate_batch(orig_groups, forg_groups, batch_size = 32):

    while True:
        orig_pairs = []
        forg_pairs = []
        gen_gen_labels = []
        gen_for_labels = []
        all_pairs = []
        all_labels = []

        for orig, forg in zip(orig_groups, forg_groups):
            orig_pairs.extend(list(itertools.combinations(orig, 2)))
            for i in range(len(forg)):
                forg_pairs.extend(list(itertools.product(orig[i:i+1], random.sample(forg, 12))))

        gen_gen_labels = [1]*len(orig_pairs)
        gen_for_labels = [0]*len(forg_pairs)

        all_pairs = orig_pairs + forg_pairs
        all_labels = gen_gen_labels + gen_for_labels
        del orig_pairs, forg_pairs, gen_gen_labels, gen_for_labels
        all_pairs, all_labels = shuffle(all_pairs, all_labels)

        k = 0
        pairs=[np.zeros((batch_size, img_h, img_w, 1)) for i in range(2)]
        targets=np.zeros((batch_size,))
        for ix, pair in enumerate(all_pairs):
            img1 = cv2.imread(pair[0], 0)
            img2 = cv2.imread(pair[1], 0)
            img1 = cv2.resize(img1, (img_w, img_h))
            img2 = cv2.resize(img2, (img_w, img_h))
            img1 = np.array(img1, dtype = np.float64)
            img2 = np.array(img2, dtype = np.float64)
            img1 /= 255
            img2 /= 255
            img1 = img1[..., np.newaxis]
            img2 = img2[..., np.newaxis]
            pairs[0][k, :, :, :] = img1
            pairs[1][k, :, :, :] = img2
            targets[k] = all_labels[ix]
            k += 1
            if k == batch_size:
                yield pairs, targets
                k = 0
                pairs=[np.zeros((batch_size, img_h, img_w, 1)) for i in range(2)]
                targets=np.zeros((batch_size,))

    return pairs, targets

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

In [ ]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [ ]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    Source: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf

    Explanation:
    When ytrue is 1, that means the sample are duplicates of each other,
    so the Euclidean distance (ypred) between their outputs must be minimized.
    So the loss is taken as the square of that Euclidean distance itself - K.square(y_pred).
    When ytrue is 0, i.e. the samples are not duplicates, then the Euclidean distance
    between them must be maximized, at least to the margin. So the loss to be minimized
    is the difference of the margin and the Euclidean distance - (margin - y_pred).
    If the Euclidean distance (ypred) is already greater than the margin,
    then nothing is to be learned, so the loss is made to be zero in
    that case by saying K.maximum(margin - y_pred, 0).
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [ ]:
def create_base_network_signet(input_shape):
    seq = Sequential()
    seq.add(Conv2D(96, kernel_size=(11, 11), activation='relu', name='conv1_1', strides=4, input_shape= input_shape))
    seq.add(BatchNormalization())
    seq.add(MaxPooling2D((2,2), strides=(1,1)))
    seq.add(Dropout(0.3))

    seq.add(Conv2D(256, kernel_size=(5, 5), activation='relu', name='conv2_1'))
    seq.add(BatchNormalization())
    seq.add(MaxPooling2D((2,2), strides=(1,1)))
    seq.add(Dropout(0.3))

    seq.add(Conv2D(512, kernel_size=(3, 3), activation='relu', name='conv3_1'))
    seq.add(MaxPooling2D((2,2), strides=(1,1)))
    seq.add(Dropout(0.3))

    seq.add(GlobalAveragePooling2D())
    seq.add(Dense(128, activation='relu'))
    return seq

In [ ]:
input_shape=(img_h, img_w, 1)
input_shape

(155, 220, 1)

In [ ]:
base_network = create_base_network_signet(input_shape)

input_a = Input(shape=(input_shape))
input_b = Input(shape=(input_shape))

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
outputs = Dense(1, activation="sigmoid")(distance)

model = Model(inputs=[input_a, input_b], outputs=outputs)

In [ ]:
batch_sz = 256
num_train_samples = 276*120 + 24 * 12 *120
num_val_samples = num_test_samples = 276*20 + 288*20
num_train_samples, num_val_samples, num_test_samples

(67680, 11280, 11280)

In [ ]:
rms = RMSprop(lr=1e-4)
model.compile(loss=contrastive_loss, optimizer=rms,metrics=[accuracy])

In [ ]:
callbacks = [
    ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.000001, verbose=1),
    ModelCheckpoint('./signet.h5', verbose=1, save_weights_only=True)
]

In [ ]:
results = model.fit(generate_batch(orig_train, forg_train, batch_sz),
                              steps_per_epoch = num_train_samples//batch_sz,
                              epochs = 10,
                              validation_data = generate_batch(orig_val, forg_val, batch_sz),
                              validation_steps = num_val_samples//batch_sz,
                              callbacks = callbacks)

Epoch 1/10
264/264 [==============================] - ETA: 0s - loss: 0.2009 - accuracy: 0.5107
Epoch 1: saving model to ./signet.h5
264/264 [==============================] - 412s 2s/step - loss: 0.2009 - accuracy: 0.5107 - val_loss: 0.2221 - val_accuracy: 0.5107 - lr: 1.0000e-04
Epoch 2/10
264/264 [==============================] - ETA: 0s - loss: 0.1738 - accuracy: 0.5336
Epoch 2: saving model to ./signet.h5
264/264 [==============================] - 399s 2s/step - loss: 0.1738 - accuracy: 0.5336 - val_loss: 0.2035 - val_accuracy: 0.5191 - lr: 1.0000e-04
Epoch 3/10
264/264 [==============================] - ETA: 0s - loss: 0.1582 - accuracy: 0.6725
Epoch 3: saving model to ./signet.h5
264/264 [==============================] - 393s 1s/step - loss: 0.1582 - accuracy: 0.6725 - val_loss: 0.1861 - val_accuracy: 0.6830 - lr: 1.0000e-04
Epoch 4/10
264/264 [==============================] - ETA: 0s - loss: 0.1461 - accuracy: 0.7747
Epoch 4: saving model to ./signet.h5
264/264 [============

KeyboardInterrupt: ignored